In [1]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
from utils.image_to_embedding import get_image_to_embedding
from utils.static_check import get_static_difference
from utils.optical_flow_check import get_optical_flow
from diffusers import AutoencoderKL
import torch

import cv2
import numpy as np
from PIL import Image
import os
from torchvision import transforms

def image_transform(image):
    transform = transforms.Compose([
        transforms.Resize((320, 240)),
        transforms.CenterCrop(240),
        transforms.ToTensor(),  
    ])
    return transform(image).unsqueeze(0)

def load_image(folder_path):
    # Sort the frames
    frames = sorted(os.listdir(folder_path))
    conv_frames = []
    
    # Convert the frames to tensor
    for frame in frames:
        image = Image.open(os.path.join(folder_path, frame)).convert('RGB')
        image = image_transform(image)
        conv_frames.append(image)
    return conv_frames

def get_model():
    model = AutoencoderKL.from_pretrained("stabilityai/sdxl-vae")
    return model

def get_inference(folder_path, model, device):
    frames = load_image(folder_path)

    # Getting static difference
    rgb_diff_max, rgb_diff_mean = get_static_difference(frames)

    # getting optical flow
    max_velocity, avg_velocity = get_optical_flow(frames)
    
    # Getting image context similarity
    frame_mse, frame_cos_sim = get_image_to_embedding(frames, model, device)

    # Getting number of frames
    num_frames = len(frames)

    return rgb_diff_max, rgb_diff_mean, max_velocity, avg_velocity, frame_mse, frame_cos_sim, num_frames

In [3]:
# Load from annotations.json
import json
with open('annotations.json', 'r') as f:
    labeled_data = json.load(f)

In [4]:
rgb_diff_maxs = []
rgb_diff_means = []
max_velocities = []
avg_velocities = []
frame_mses = []
frame_cos_sims = []
num_frames_list = []
labels = []

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = get_model().to(device)

for i, path in enumerate(labeled_data.keys()):
    rgb_diff_max, rgb_diff_mean, max_velocity, avg_velocity, frame_mse, frame_cos_sim, num_frames = get_inference(path, model, device)
    rgb_diff_maxs.append(rgb_diff_max)
    rgb_diff_means.append(rgb_diff_mean)
    max_velocities.append(max_velocity)
    avg_velocities.append(avg_velocity)
    frame_mses.append(frame_mse)
    frame_cos_sims.append(frame_cos_sim)
    num_frames_list.append(num_frames)
    labels.append(labeled_data[path])
    print(f'Enumerated {i+1} paths')

Enumerated 1 paths
Enumerated 2 paths
Enumerated 3 paths
Enumerated 4 paths
Enumerated 5 paths
Enumerated 6 paths
Enumerated 7 paths
Enumerated 8 paths
Enumerated 9 paths
Enumerated 10 paths
Enumerated 11 paths
Enumerated 12 paths
Enumerated 13 paths
Enumerated 14 paths
Enumerated 15 paths
Enumerated 16 paths
Enumerated 17 paths
Enumerated 18 paths
Enumerated 19 paths
Enumerated 20 paths
Enumerated 21 paths
Enumerated 22 paths
Enumerated 23 paths
Enumerated 24 paths
Enumerated 25 paths
Enumerated 26 paths
Enumerated 27 paths
Enumerated 28 paths
Enumerated 29 paths
Enumerated 30 paths
Enumerated 31 paths
Enumerated 32 paths
Enumerated 33 paths
Enumerated 34 paths
Enumerated 35 paths
Enumerated 36 paths
Enumerated 37 paths
Enumerated 38 paths
Enumerated 39 paths
Enumerated 40 paths
Enumerated 41 paths
Enumerated 42 paths
Enumerated 43 paths
Enumerated 44 paths
Enumerated 45 paths
Enumerated 46 paths
Enumerated 47 paths
Enumerated 48 paths
Enumerated 49 paths
Enumerated 50 paths
Enumerate

In [6]:
import pandas as pd

dataset = pd.DataFrame({
    "max_rgb_diff": rgb_diff_maxs,
    "mean_rgb_diff": rgb_diff_means,
    "max_velocity": max_velocities,
    "mean_velocity": avg_velocities,
    "mse": frame_mses,
    "cos_sim": frame_cos_sims,
    "no_frames": num_frames_list,
    "label": labels
})
dataset.head()

,max_rgb_diff,mean_rgb_diff,max_velocity,mean_velocity,mse,cos_sim,no_frames,label
0,696.234253,265.932800,12.473114,7.542836,11.076690,0.748877,60,1
1,142.426315,103.156654,0.620393,0.525993,3.880109,0.924669,7,1
2,1034.846191,390.833527,5.756709,4.963755,13.054274,0.721435,59,1
3,104.106194,20.794767,0.917800,0.596388,2.001047,0.976434,26,1
4,5661.599121,367.071655,9.760819,5.455320,11.926335,0.748189,28,0


In [7]:
dataset.corr()

,max_rgb_diff,mean_rgb_diff,max_velocity,mean_velocity,mse,cos_sim,no_frames,label
max_rgb_diff,1.000000,0.558016,0.296977,0.158794,0.276107,-0.173854,0.092808,-0.288155
mean_rgb_diff,0.558016,1.000000,0.324095,0.340854,0.822956,-0.611652,-0.064194,0.023618
max_velocity,0.296977,0.324095,1.000000,0.890355,0.401765,-0.358556,0.201489,-0.038308
mean_velocity,0.158794,0.340854,0.890355,1.000000,0.477395,-0.432766,0.138010,0.048072
mse,0.276107,0.822956,0.401765,0.477395,1.000000,-0.874607,-0.052466,0.184298
cos_sim,-0.173854,-0.611652,-0.358556,-0.432766,-0.874607,1.000000,0.045450,-0.149141
no_frames,0.092808,-0.064194,0.201489,0.138010,-0.052466,0.045450,1.000000,0.084073
label,-0.288155,0.023618,-0.038308,0.048072,0.184298,-0.149141,0.084073,1.000000


In [8]:
dataset.describe()

,max_rgb_diff,mean_rgb_diff,max_velocity,mean_velocity,mse,cos_sim,no_frames,label
count,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000
mean,3181.503418,522.466919,7.323443,4.223952,11.293567,0.854618,37.562016,0.616279
std,4735.148438,701.674683,5.824468,3.921226,9.689132,0.132447,29.032218,0.487236
min,0.109219,0.019441,0.008891,0.007581,0.055443,0.093587,4.000000,0.000000
25%,342.513474,87.631962,3.136222,1.423700,3.899280,0.792531,20.000000,0.000000
50%,1453.235596,271.241714,6.422338,3.434513,8.828903,0.897889,31.000000,1.000000
75%,4021.528015,653.851608,10.090562,6.312151,15.612548,0.953994,47.000000,1.000000
max,34677.078125,4575.578125,44.851021,30.878588,49.426112,0.999651,256.000000,1.000000


In [9]:
X = dataset.drop(columns=["label"])
y = dataset["label"]

In [10]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Creating an XGBoost classifier
model = xgb.XGBClassifier()

# Training the model
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="logloss", eval_set=eval_set, verbose=False)

y_pred_train = model.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
print("Train Accuracy:", accuracy_train)

# Making predictions on the test set
y_pred = model.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Train Accuracy: 0.9854368932038835
Accuracy: 0.8461538461538461


In [11]:
import pickle

# Save the trained model to a file
filename = 'xgboost_model.pkl'
pickle.dump(model, open(filename, 'wb'))

In [12]:
# TO TEST LOADING THE SAVED MODEL

# Load the saved model from file
classifier_model = pickle.load(open(filename, 'rb'))

# Use the loaded model for predictions
y_pred_loaded = classifier_model.predict(X_test)

# Evaluate the loaded model
accuracy_loaded = accuracy_score(y_test, y_pred_loaded)
print("Accuracy of loaded model:", accuracy_loaded)

Accuracy of loaded model: 0.8461538461538461
